## 1. Handling Data

In [39]:
#importing all the neccesery libs. 
import csv
import math
import random


In [40]:
#defining Load CSV File
def loadCsv(filename):
    lines = csv.reader(open(r'C:/Users/win/Desktop/diabetes.csv')) #taking each row in the CSV file 
    dataset = list(lines)#putiing each line as a element in the list in order to access it easily
    for i in range(len(dataset)):
            dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [41]:
def splitDataset(dataset,splitRatio):
    trainSize = int(len(dataset)*splitRatio)
    trainSet = []
    copy = list(dataset)
    while(len(trainSet)<trainSize): #we know why we are doing this
        index = random.randrange(len(copy)) #taking random sample for training
        trainSet.append(copy.pop(index)) # putting it in the trainset and reaming is testSet
    return [trainSet,copy] #now the reaming copy is nothing but our testing set



## 2. Summarizing Data

In [42]:
def separatedByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if(vector[-1] not in separated):
            separated[vector[-1]]=[]   #if not in the part of class
        separated[vector[-1]].append(vector) #merge all the things if part of class will get into Class A and Class B will be blank
    return separated

In [43]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))  #simply taking avg of numbers
        

In [44]:
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1) #taking the variaance
    return math.sqrt(variance)

In [45]:
def summarize(dataset):
    summaries = [(mean(attribute),stdev(attribute)) for attribute in zip(*dataset)] #we are getting summary of dataset as mean and variance
    del summaries[-1]
    return summaries

In [46]:
def summarizeByClass(dataset): #getting summeries by class in this function
    separated = separatedByClass(dataset)
    summaries = {}
    for classValue,instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries
        

##  3. Making a prediction

In [47]:
def calculateProbability(x,mean,stdev): #using the formula for normal distrubtion we are getting Likelihood here 
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1/(math.sqrt(2*math.pi)*stdev))*exponent

In [48]:
def calculateClassProbability(summaries,inputVector): #we are doing the denomintor part p(x) <- this thing from Bayes's formula
    probabilities = {}
    for classValue, classSumaries in summaries.items():
        probabilities[classValue] =1
        for i in range(len(classSumaries)):
            mean,stdev = classSumaries[i]
            x  = inputVector[i]
            probabilities[classValue]*= calculateProbability(x,mean,stdev)
        return probabilities
    

In [49]:
def predict(summaries,inputVector): #we are getting which is bestlable we got
    probabilities =  calculateClassProbability(summaries,inputVector)
    bestLabel, bestProb = None,-1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [50]:
def getPrediction(summaries,testSet): #We are getting prediction of all the test set here
    prediction =[]
    for i in range(len(testSet)):
        result = predict(summaries,testSet[i])
        prediction.append(result)
    return prediction

## 4. Getting Accuracy 

In [64]:
def getAccuracy(testSet,prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == prediction[x]:
            correct+=1
    return (correct/float(len(testSet)))*100

In [65]:
def main():
    filename='PIMA Dataset'
    splitRatio = 0.67
    dataset= loadCsv(filename)
    trainingSet,testSet = splitDataset(dataset,splitRatio)
    print('Split {0} rows into train={1} and test={2} rows '.format(len(dataset),len(trainingSet),len(testSet)))
    #preparing Model
    summaries = summarizeByClass(trainingSet)
    #testing
    predictions = getPrediction(summaries,testSet)
    accuracy = getAccuracy(testSet,predictions)
    print('Accuracy: {0} %'.format(accuracy))
main()

Split 768 rows into train=514 and test=254 rows 
Accuracy: 67.32283464566929 %
